## Histogrammes de gradients orientés
[Dalal, N. and Triggs, B., “Histograms of Oriented Gradients for Human Detection,” IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2005, San Diego, CA, USA.](https://ieeexplore.ieee.org/document/1467360)

Les descripteurs de caractéristiques de gradient orienté histogramme (HOG) sont populaires pour la détection d'objets. L'algorithme comporte les étapes suivantes :

### Prétraitement
Dans la première étape de l'algorithme, une normalisation de l'image est effectuée afin de réduire les effets de sa luminosité sur le vecteur de traits descriptifs.

En pratique, on utilise soit la compression gamma (loi de puissance), soit le calcul de la racine carrée ou du logarithme de chaque canal de couleur.

La force de la **texture de l'image** est généralement **proportionnelle** à l'**éclairage local de l'image**. Cette compression permet donc de réduire les effets des variations locales d'ombre et d'éclairage.

Si l'image est en couleur, la normalisation est appliquée à chacun de ses canaux.

### Calcul des dégradés de l'image
Dans la deuxième étape, les **pentes de l'image** sont calculées (c'est-à-dire la première
dérivées selon les directions x et y). Le masque qui a fourni les meilleurs résultats pour le calcul des gradients est le ** unidimensionnel
masque [-1,0,1]**.

Les gradients fournissent des informations sur l'existence
bords et la forme des objets représentés. Dans certaines variantes de l'algorithme, en plus des premières dérivées sont également calculées
les dérivées secondes de l'image. Dans le cas d'images en couleur
les pentes des trois canaux sont calculées et la plus grande est sélectionnée
valeur.



### Calculer l'histogramme des pentes dans les cellules
Dans la troisième étape, l'image est divisée en cellules.

Chaque cellule est constituée d'un nombre spécifique et prédéterminé de pixels et peut être rectangulaire (par exemple 8x8) ou radiale.

Pour chaque cellule, un histogramme unidimensionnel est créé qui enregistre les pentes de tous les pixels qui lui appartiennent. L'inscription a lieu à des adresses spécifiques.


Par exemple, les cellules d'image 8x8 contiennent 8x8x3 = 192 pixels.
Le gradient de chacune de ces cellules se compose de 2 valeurs (largeur et angle) pour chaque pixel soit 8x8x2 = 128 valeurs.
Chacun de ces segments avec les 128 valeurs sera représenté sous la forme d'un histogramme à 9 cases qui sera stocké dans un tableau à 9 colonnes et sera robuste au bruit.

![alt text](https://drive.google.com/uc?id=1lc8odEpEpIzZfX83cFcJBOCWY1afhOhG)

En particulier, selon le nombre prédéterminé d'intervalles utilisés, un histogramme avec des orientations spécifiques est créé.

La plage de l'histogramme peut aller de $0^o-180^o$
(dégradé non signé) ou de $0^o-360^o$ (dégradé signé). Par exemple, si l'espace $0^o-180^o$
est subdivisé par $20^o$, un histogramme avec neuf directions de pente possibles est créé, tandis que si l'espace $0^o-360^o$ est divisé, un histogramme est créé avec 18 directions possibles.

Il a été démontré empiriquement que les gradients non étiquetés fonctionnent mieux que les gradients étiquetés pour la détection des piétons.

L'étape suivante consiste à créer un histogramme de gradient dans ces cellules 8 × 8. L'histogramme contient 9 cases correspondant aux angles 0, 20, 40 … 160. La figure ci-dessous illustre le processus. Nous considérons l'amplitude et la direction de la pente de la même cellule 8 × 8 que dans la figure précédente.

Pour chaque pixel, la direction (ou direction) du dégradé est calculée, et il vote avec un certain poids pour l'intervalle correspondant auquel il appartient.

Concentrons-nous d'abord sur le pixel entouré de bleu. Il a un angle (direction) de 80 degrés et une magnitude de 2. Il ajoute donc 2 au 5e bac. La pente au niveau du pixel entouré en rouge a un angle de 10 degrés et une magnitude de 4. Comme 10 degrés est à mi-chemin entre 0 et 20, le vote du pixel est réparti également entre les deux bacs.

![](https://drive.google.com//uc?id=1mCnmWIL1T98W7aPLvbasxvEwMWkNL4iH)



Si l'angle est supérieur à 160 degrés, il est compris entre 160 et 180, et nous savons que l'angle s'enroule autour de faire 0 et 180 équivalent. Ainsi, dans l'exemple ci-dessous, le pixel avec un angle de 165 degrés contribue proportionnellement au bac de 0 degré et au bac de 160 degrés.


Pour $θ=165^o$, la mesure doit être distribuée en fonction de la distance de la phase aux centres des cases voisines. À partir de 180 $^o$, $θ$ est à 15 $^o$, donc dans le bac, ils sont proportionnels à $\frac{15}{20}=\frac{3}{4}$ de la mesure (85 $ \cdot \frac{3}{ 4})=$63.75) tandis que le bac suivant correspond à $\frac{5}{20}= \frac{1}{4}$ (($85 \cdot\frac{1}{4} )= 21,25 $))

![alt text](https://drive.google.com/uc?id=1tgfxD0KAg-UmE1FVC9F3HnN644lnYQ98)

Les contributions de tous les pixels dans les cellules 8 × 8 sont ajoutées pour créer l'histogramme 9-bin.

![alt text](https://drive.google.com/uc?id=1zgFkfQVzIKe3Q2gFt4rgPuL_9MVq8vev)

L'histogramme résultant de la cellule après ce processus semble avoir de nombreuses valeurs proches de 0 et 180 degrés, ce qui est juste une autre façon de dire que la cellule est inclinée vers le haut ou vers le bas.

### Normalisation de bloc 16 × 16

Dans la quatrième étape, les histogrammes résultants sont normalisés
pour une plus grande tolérance aux ombres et aux changements brusques de luminosité.

Les dégradés d'une image sont sensibles à la luminosité. Si, par exemple, nous réduisons de moitié la largeur des pentes (/2 de toutes les valeurs), les valeurs de l'histogramme seront également divisées par deux. C'est pourquoi nous sommes poussés à la normalisation pour éviter les variations d'éclairage.

*(Exemple de normalisation vectorielle RVB : soit le vecteur $v_1=[ 128, 64, 32 ]$ En appliquant la norme L2, nous aurons $[0.87, 0.43, 0.22]$. Soit le vecteur $v_2=2\cdot v_1=[ 256, 128, 64 ]$.En appliquant la norme L2 à $v_2$ nous obtiendrons $
[0.87, 0.43, 0.22]$ το οποίο είναι ίδιο με το κανονικοποιημένο διάνυσμα του $v_1$. Άρα με την κανονικοποίηση αφαιρείται η κλιμάκωση). Nous pourrions donc normaliser l'histogramme 9 × 1 de la même manière que nous avons normalisé le vecteur 3 × 1 *

![](https://drive.google.com/uc?id=1EyPRgx2pwBRw3X7QzvQ0MhMJUUasZx-W)

Au lieu de normaliser l'histogramme 9 × 1 résultant de chaque cellule, l'algorithme HOG normalise une taille de bloc plus grande de 16 × 16. Un bloc 16 × 16 a 4 histogrammes qui peuvent être combinés pour former un vecteur 36 × 1 qui est normalisé. La fenêtre est ensuite déplacée de 8 pixels et le vecteur normalisé 36 × 1 suivant dans cette fenêtre est calculé et le processus est répété jusqu'à ce que tous les blocs soient obtenus.

Ainsi chaque cellule est partagée entre plusieurs blocs, mais ses normalisations sont dépendantes des blocs et donc différentes. Ainsi, la cellule apparaît plusieurs fois dans le descripteur de sortie avec des normalisations différentes. Cela peut sembler inutile, mais cela améliore les performances.

### Calcul du vecteur caractéristique HOG
L'étape finale rassemble les descripteurs HOG de tous les blocs 16x16 dans un vecteur qui peut être utilisé dans diverses applications d'image (classification, détection d'objets,...).

Dans l'image ci-dessus, nous remarquons que nous avons 7x15=105 blocs. Chaque bloc 16x16 est représenté par un vecteur d'histogramme 36x1. Donc pour cette image particulière, un vecteur final de dimension 105x36x1 = 3780x1 est obtenu

In [ ]:
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure


image = data.astronaut()


fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), block_norm="L2", feature_vector="True", visualize=True, multichannel=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()
print(fd)

Output hidden; open in https://colab.research.google.com to view.